![Winning Variant!](https://static.winningvariant.com/images/brand/winning-variant-header-small.jpg)

# Getting Started with Winning Variant

The following notebook walks you through a basic example of how to configure and use Winning Variant, the _only_ Snowflake-native experimentation engine.

## Use application

Update the following snippet to use the app as it's named in your account...

In [ ]:
USE WINNING_VARIANT_DEMO_NOTEBOOK;

## Create a Lab

All experiments live within a [Lab](https://docs.winningvariant.com/resources/labs), with is a workspace that can be assigned to a department, team, or individual. This sample lab is simply called "Data Science" for an internal team that builds and optimizes models and algorithms.

In [ ]:
CALL experimentation.apply_resource($$
schemaVersion: 1
kind: lab
metadata:
  id: datascience
  status: active
  name: Data Science
  description: Internal Data Science Team
$$);

## Create a Subject Type

A [Subject Type](https://docs.winningvariant.com/resources/subjects) is a type of identifier that can enter an experiment. Think of this as the thing that will be tested. Common use cases are "anonymous IDs" or a known "customer ID". In our example, we'll be testing a search algorithm against all users on our site, so we'll create an `anonymous_id` subject type that is presumably a cookie value stored in the user's browser.

In [ ]:
CALL experimentation.apply_resource($$
schemaVersion: 1
kind: subjecttype
metadata:
  id: anonymous_id
  status: active
  name: Anonymous ID stored in browser cookie
spec:
  matchType: UUID
$$);

## Create Experiment

Now that we have a Lab for the experiment to reside in and a Subject Type to experiment on, we can create an experiment. This experiment is to test a new version of a search algorithm that powers a product search on a website. Ultimately, we're interested in testing if this new version drives more sales.

A few callouts about this experiment:

* We specify the `datascience` lab as its parent.
* We specify our `anonymous_id` subject type as the one to use when creating assignments.
* This initial release has just two variants: `control` and `v2`, launched with a 50/50 split in the initial cohort.

**Note about statuses**
 
Each resource has a `status` field to denote its lifecycle stage (e.g., `draft` vs `active`). For the sake of simplicity, this experiment is created as `active`, which will make it immediately available for generating assignments. In the real world, we'd like create it as a `draft` while we build it out to have it ready for launch and would set it to `active` only when we're ready to launch the experiment.


In [ ]:
CALL experimentation.apply_resource($$
schemaVersion: 1
kind: experiment
metadata:
  id: search-v2
  status: active
  name: Search Algorithm v2
  description: "Test the new version of our internal search algorithm against purchases."
  parentKind: lab
  parentId: datascience
spec:
  subjectType: anonymous_id
  hypothesis: "The new version of the search algorithm will increase purchases and return results faster, on average."
  variants:
    - id: control
      isControl: true
      name: Control
      description: "Current version of the search algorithm."
    - id: v2
      isControl: false
      name: "v2"
      description: "v2 search algorithm."
  cohorts:
    - index: 1
      variants:
      - variant: control
        split: 0.5
      - variant: v2
        split: 0.5
$$);

## View Resources

Before creating assignments, let's take a look at all of the resources we created. We'll do this quickly by performing a union on our [dimension tables](https://docs.winningvariant.com/analytics/dimensions). We could also get resources of a given type using [`experimentation.list_resources()`](https://docs.winningvariant.com/resources/manage#list-resources).

In [ ]:
SELECT 'EXPERIMENT' as kind, id, name, status, description FROM experimentation.experiments
UNION ALL
SELECT 'LAB' as kind, id, name, status, description FROM experimentation.labs
UNION ALL
SELECT 'SUBJECT_TYPE' as kind, id, name, status, description FROM experimentation.subject_types;

## Create assignments

Now that everything is created, we'll make some [assignments](https://docs.winningvariant.com/assignments/overview) in our experiment. We define an "assignment" as the variant a subject is assigned within an experiment. This represents the variant that the subject/user/etc should see or experience.

We'll use the [`experimentation.create_assignment()`](https://docs.winningvariant.com/assignments/manage#get-or-make-assignments) procedure, which takes the subject type, subject ID, and experiment ID.

The `anonymous_id` subject type was set up as a UUID, so we'll use a series of random UUIDs to make assignments with.

In [ ]:
SELECT experimentation.create_assignment('anonymous_id', '77e3491c-91a0-4e0e-8485-53d690032a31', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '572bf683-3cc9-461c-bd8a-91e103ef79ef', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '8da7979d-08e5-47b9-a203-1e64557e1e57', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '5cb802c6-483c-4add-a2a6-5c0893a085a8', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '48056be6-af89-468a-a0f8-d003a4a9963c', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '58321770-51cc-4ed9-8b7b-4791f8929584', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '0da6ab28-4a73-4bea-852f-08659a197c12', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '0a1bc9d1-a2c6-464a-89a8-b1dd186172ae', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', '0afe2d41-018b-481e-b726-ab0232ee7fe8', 'search-v2');
SELECT experimentation.create_assignment('anonymous_id', 'a268c96d-9235-4137-bbd2-bb763a73cf71', 'search-v2');

SELECT * FROM experimentation.assignments;